In [ ]:
!pip install -U sentence-transformers
# https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

In [ ]:
import sys
sys.path.append("../src")

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from etf_transformations import *


In [ ]:
base_data = pd.read_csv("../data/preprocessed/separate_stocks_finbert.csv")
base_data.head(2)

In [ ]:
base_data2 = pd.read_parquet("../data/preprocessed/separate_stocks_finbert.parquet")
base_data2.head(2)

In [ ]:
base_data['row_id'] = base_data.index
base_data.head()

In [ ]:
base_data.columns

In [ ]:
base_data.nunique()

In [ ]:
# mini test from the transformers website

sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings, embeddings.shape)

In [ ]:
sentences = base_data["Headlines"].astype(str).tolist()

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = model.encode(
    sentences,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True
)


In [ ]:
embeddings_as_df = pd.DataFrame(embeddings, columns=[f"emb_{i}" for i in range(embeddings.shape[1])])

In [ ]:
embeddings_as_df.head()

In [ ]:
i, j = 0,1  # indeksy w df

v1 = embeddings_as_df.loc[i, [c for c in embeddings_as_df.columns if c.startswith("emb_")]].values.reshape(1, -1)
v2 = embeddings_as_df.loc[j, [c for c in embeddings_as_df.columns if c.startswith("emb_")]].values.reshape(1, -1)

cosine_similarity(v1, v2)[0, 0]


In [ ]:
embeddings_as_df["row_id"] = base_data["row_id"].values

In [ ]:
# TU WTEDY TRZEBA ZROBIC MERGE

In [ ]:
# CONFIDENCE DISTRIBUTION

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

conf = base_data["finbert_confidence"].astype(float)

print("N rows:", len(base_data))
print("NaNs in finbert_confidence:", conf.isna().sum(), f"({conf.isna().mean():.2%})")
print("Describe:")
print(conf.describe())

print("Quantiles:")
print(conf.quantile([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]))


plt.figure()
plt.hist(conf.dropna().values, bins=50)
plt.title("finbert_confidence distribution")
plt.xlabel("finbert_confidence")
plt.ylabel("count")
plt.show()

sector_cols = [c for c in ["XLE","XLF","XLK","XLV","XLY"] if c in base_data.columns]

for s in sector_cols:
    c = base_data.loc[base_data[s] == 1, "finbert_confidence"].astype(float)
    if len(c.dropna()) == 0:
        print(f"\n{s}: no non-NaN confidence values")
        continue

    print(f"{s}")
    print("N headlines:", len(c), " NaNs:", c.isna().sum())
    print("Describe:")


